In [ ]:
Demo : You can create a Linear Regression Model to predict house prices based on:
    - Square foot(Size)
    - Number of Bedrooms
    - Age of the house
    
Steps for ML Pipeline
Step 1: Create an in  memory dataset
Step 2: Perform Feature  Engineering ( Convert feature into vector)
Step 3: Train a Linear Regression Model
Step 4: Make prediction
Step 5: Evaluate model Performance

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import col, when

spark = SparkSession.builder.appName("HousePricePrediction").getOrCreate()

data = [
    (1500, 3, 20, 300000),
    (2000, 4, 15, 400000),
    (2500, 4, 10, 500000),
    (1800, 3, 5, 350000),
    (3000, 5, 2, 600000),
    (1200, 2, 30, 200000),
]

columns = ["Size", "Bedrooms", "Age", "Price"]
df = spark.createDataFrame(data, columns)

assembler = VectorAssembler(inputCols=["Size", "Bedrooms", "Age"], outputCol="features")
df = assembler.transform(df).select("features", col("Price").alias("label"))

train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

lr = LinearRegression(featuresCol="features", labelCol="label")
model = lr.fit(train_df)

predictions = model.transform(test_df)
predictions.select("features", "label", "prediction").show()

print(f"Intercept: {model.intercept}")
print(f"Coefficients: {model.coefficients}")

spark.stop()


25/02/13 11:04:42 WARN Instrumentation: [a9518617] regParam is zero, which might cause numerical instability and overfitting.
25/02/13 11:04:45 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
25/02/13 11:04:45 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
25/02/13 11:04:45 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
25/02/13 11:04:45 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


+-----------------+------+------------------+
|         features| label|        prediction|
+-----------------+------+------------------+
|[2500.0,4.0,10.0]|500000|502941.17647058814|
|[1200.0,2.0,30.0]|200000| 248039.2156862743|
+-----------------+------+------------------+

Intercept: -34313.72549019366
Coefficients: [215.68627450980333,-2941.176470588368,980.3921568626912]
